In [3]:
import os

In [4]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_classes: int

In [8]:
from Facial_Review_System.constants import *
from Facial_Review_System.utils.common import read_yaml, create_directories

In [9]:
#this class for base model configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [10]:
import os
import zipfile
import gdown
from Facial_Review_System import logger
from Facial_Review_System.utils.common import get_size
import tensorflow as tf

In [22]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        
        self.model = tf.keras.models.load_model("artifacts/prepare_base_model/base_model.h5")

        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def _prepare_full_model(model, classes):
        
        for layer in model.layers:
            model.trainable = False
            
        Input = tf.keras.layers.Input(shape=(224, 224, 3))
        
        vgg = model(Input)
        
        flatten_in = tf.keras.layers.Flatten()(vgg)
        
        dense1 = tf.keras.layers.Dense(
            units=512,
            activation="relu"
        )(flatten_in)
        
        dense2 = tf.keras.layers.Dense(
            units=206,
            activation="relu"
        )(dense1)
        
        dropout = tf.keras.layers.Dropout(0.25)(dense2)
        
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(dropout)
        
        full_model = tf.keras.models.Model(
            inputs=Input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
        
    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)

In [23]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-04-26 17:23:45,975: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-26 17:23:45,981: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-26 17:23:45,986: INFO: common: created directory at: artifacts]
[2024-04-26 17:23:45,988: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-04-26 17:23:48,252: WARNING: legacy_h5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
[2024-04-26 17:23:48,258: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


Model: "functional_185"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 1, 1, 2622)     │   145,002,878 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2622)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     1,342,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 206)            │       105,678 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 206)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 7)              │         1,449 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 146,452,981 (558.67 MB)

 Trainable params: 1,450,103 (5.53 MB)

 Non-trainable params: 145,002,878 (553.14 MB)

[2024-04-26 17:23:50,101: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
